# Deep learning framework example: Named Entity Recognition

This notebook demonstrates how to use the deeplearning API to train and test the model on the [CoNNL 2003 dataset](https://www.clips.uantwerpen.be/conll2003/ner/).  The task is to identify named entities (i.e. people, organizations etc).

**Important**: Please see the Movie Review notebook example in the `zensols.movie` API first, as it contains more explaination of how the framework is used.  The purpose of this notebook is to run the MNIST dataset and visualize the results.

In [ ]:
# set up notebook environment
import sys
from pathlib import Path
app_root_dir = Path('..')
# add the example to the Python library path
sys.path.append(str(app_root_dir / 'src'))
# add the deepnlp path
sys.path.append('../../../src/python')

## Configure and create the app specific facade

Now that the interpreter environment is set up, we can import local packages.

In [ ]:
from zensols.deeplearn import TorchConfig
from zensols.deeplearn.cli import JupyterManager
from ner import CliFactory

# reset random state for consistency of each new test that uses this function
TorchConfig.init()

mng = JupyterManager(
    allocation_tracking=True,
    cli_class=CliFactory,
    factory_args={'root_dir': app_root_dir},
    cli_args_fn=lambda model: ['-c', str(app_root_dir / 'models' / f'{model}.conf')])

reset_epochs = False

## Print information about 

Use the factory to create the model executor.  The `write` method gives statistics on the data set that is configured on the executor.

In [ ]:
from zensols.config import Writable
# set indention level for human readable (pretty print) output
Writable.WRITABLE_INDENT_SPACE = 2
facade = mng.create_facade('glove50')
facade.write()

## Train and test the model

Train and test the model with the default (low) number of epochs to make sure everything is working.

In [ ]:
facade.epochs = 2
mng.run()

## Tune hyperparameters

Set model parameters to get a feel for where they need to be before changing features.  Start with Glove 50 dimensional word embeddings with a learning rate of 0.01 and 20 epochs.

In [ ]:
facade.learning_rate = 0.01
if reset_epochs:
    facade.epochs = 20
mng.run()

# Glove 300 embeddings

Next we use the same learning rate, but switch to the 300 dimension version of the embeddings.  The number of epochs is reduced because I have run the test before I know at what epoch the validation loss converges.  Since the model is saved only when the validation loss decreases, we early stop at 8 epochs.

In [ ]:
if reset_epochs:
    facade.epochs = 8
facade.embedding = 'glove_300_embedding'
mng.run()

# Word2vec Embeddings

Now we switch to the Google 300D word2vec pretrained vectors using 12 epochs, even though it has converged at 9 epochs previously.

In [ ]:
if reset_epochs:
    facade.epochs = 12
facade.embedding = 'word2vec_300_embedding'
mng.run()

# BERT Embeddings

Now we test with Bert context aware frozen (not trainable) embeddings using 10 epochs.  We must empty the `net_settings` attributes, which are the lingustic features, since Bert tokenizes using the word piece algorithm and the tensor shapes will not align.  We'll address this later.

In [ ]:
facade = mng.create_facade('transformer-fixed')
facade.epochs = 1
if reset_epochs:
    facade.epochs = 10
facade.net_settings.add_attributes = ()
mng.run()

In [ ]:
facade.net_settings.add_attributes = ('syns_expander', 'tags_expander')
mng.run()

In [ ]:
if 0:
    mng.show_leaks()
    mng.cleanup()
else:
    facade = mng.create_facade('transformer-trainable')
    facade.model_settings.batch_limit = 1
    facade.epochs = 1
    #logging.getLogger('zensols.deepnlp.vectorize.layer').setLevel(logging.DEBUG)
    mng.run()

In [ ]:
#from zensols.persist import Deallocatable
#Deallocatable._print_undeallocated(only_counts=False, include_stack=True)
#mng.deallocate()
#mng.cleanup()
#mng.show_leaks()
#a = next(iter(Deallocatable.ALLOCATIONS.values()))[0]
#print(type(a), a)